# Week 6: SQL and NoSQL

## Exercise 5.1 (simple query)
**The customer with customerID ALFKI has made a number of orders containing some products. Query for, and return, all orders made by ALFKI and the products they contain. Return at least the product id and the name of the products.**


### Expected outcome

#10643 with Rössle Sauerkraut, Spegesild and Chartreuse verte
#10692 with Vegie-spread
#10702 with Aniseed Syrup and Lakkalikööri
#10835 with Raclette Courdavault and Original Frankfurter grüne Soße
#10952 with Grandma's Boysenberry Spread and Rössle Sauerkraut
#11011 with Escargots de Bourgogne and Flotemysost


### Mongo db

In [12]:
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
from pprint import pprint

mongodb_url = "mongodb://localhost:27017"

# connect to MongoDB
client = MongoClient(mongodb_url)
db=client.Northwind
# Issue the serverStatus command and print the results
serverStatusResult=db.command("serverStatus")
#pprint(serverStatusResult)

In [11]:
# Get the collections from the Northwind database
collection = db.Northwind
# print out all of the collections
pprint(db.collection_names(include_system_collections=False))

[u'suppliers',
 u'territories',
 u'regions',
 u'northwind',
 u'categories',
 u'products',
 u'employee-territories',
 u'orders',
 u'customers',
 u'shippers',
 u'order-details']


In [107]:
orders = db["order-details"].aggregate([
        {
            "$lookup":
                {
                    "from" : "orders",
                    "localField" : "OrderID",
                    "foreignField" : "OrderID",
                    "as" : "order"
                }
       },
        { 
            "$match" : { "order.CustomerID":"ALFKI" }
        },
        {
            "$lookup":
                {
                    "from" : "products",
                    "localField" : "ProductID",
                    "foreignField" : "ProductID",
                    "as" : "productInfo"
                }
       },
        {
            "$project" : {"_id":0 ,"OrderID": 1, "ProductID": 1, "order":{"CustomerID":1}, "productInfo":{"ProductName":1}}
        },
        { 
            "$unwind" : "$order" 
        },
        { 
            "$unwind" : "$productInfo" 
        },
        {
            "$project" : {"_id":0 ,"OrderID": 1, "ProductID": 1, "CustomerID":"$order.CustomerID", "ProductName":"$productInfo.ProductName"}
        }
    ])

for order in orders:
    pprint(order)

{u'CustomerID': u'ALFKI',
 u'OrderID': 10643,
 u'ProductID': 28,
 u'ProductName': u'R\xf6ssle Sauerkraut'}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10643,
 u'ProductID': 46,
 u'ProductName': u'Spegesild'}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10643,
 u'ProductID': 39,
 u'ProductName': u'Chartreuse verte'}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10692,
 u'ProductID': 63,
 u'ProductName': u'Vegie-spread'}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10702,
 u'ProductID': 76,
 u'ProductName': u'Lakkalik\xf6\xf6ri'}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10702,
 u'ProductID': 3,
 u'ProductName': u'Aniseed Syrup'}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10835,
 u'ProductID': 59,
 u'ProductName': u'Raclette Courdavault'}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10835,
 u'ProductID': 77,
 u'ProductName': u'Original Frankfurter gr\xfcne So\xdfe'}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10952,
 u'ProductID': 6,
 u'ProductName': u"Grandma's Boysenberry Spread"}
{u'CustomerID': u'ALFKI',
 u'OrderID': 10952,


### Sql lite db

In [48]:
# start with the sql query
import sqlite3

con = sqlite3.connect('/Users/GretarAtli/Documents/GitHub/Dtu/Dtu-ToolsForBigData/week_six/northwind.db')
con.text_factory = str

with con:
    
    cur = con.cursor()
   
    cur.execute(""" 
        SELECT ode.OrderID, o.CustomerID, ode.ProductID, p.ProductName
        FROM "Order Details" ode
        INNER JOIN  orders o on ode.OrderID = o.OrderID
        INNER JOIN Products p on ode.ProductID = p.ProductID
        WHERE CustomerId = "ALFKI" 
        """)
    
    print "(OrderID, CustomerID, ProductID, ProductName)"
    print "---------------------------------------------"
    
    data = cur.fetchall()
    for d in data:
        print d

print "\nALFKI purchased {} items".format(len(data))

(OrderID, CustomerID, ProductID, ProductName)
---------------------------------------------
(10643, 'ALFKI', 28, 'R\xf6ssle Sauerkraut')
(10643, 'ALFKI', 39, 'Chartreuse verte')
(10643, 'ALFKI', 46, 'Spegesild')
(10692, 'ALFKI', 63, 'Vegie-spread')
(10702, 'ALFKI', 3, 'Aniseed Syrup')
(10702, 'ALFKI', 76, 'Lakkalik\xf6\xf6ri')
(10835, 'ALFKI', 59, 'Raclette Courdavault')
(10835, 'ALFKI', 77, 'Original Frankfurter gr\xfcne So\xdfe')
(10952, 'ALFKI', 6, "Grandma's Boysenberry Spread")
(10952, 'ALFKI', 28, 'R\xf6ssle Sauerkraut')
(11011, 'ALFKI', 58, 'Escargots de Bourgogne')
(11011, 'ALFKI', 71, 'Flotemysost')

ALFKI purchased 12 items


## Exercise 5.2 (harder query)
**Get all orders (with products) made by ALFKI that contain at least 2 different product types.**

### Expected outcome

#10643 with Rössle Sauerkraut, Spegesild and Chartreuse verte
#10702 with Aniseed Syrup and Lakkalikööri
#10835 with Raclette Courdavault and Original Frankfurter grüne Soße
#10952 with Grandma's Boysenberry Spread and Rössle Sauerkraut
#11011 with Escargots de Bourgogne and Flotemysost


### Sql lite db

In [49]:
# start with the sql query
import sqlite3

con = sqlite3.connect('/Users/GretarAtli/Documents/GitHub/Dtu/Dtu-ToolsForBigData/week_six/northwind.db')
con.text_factory = str

with con:
    
    cur = con.cursor()
   
    cur.execute(""" 
        SELECT ode.OrderID, o.CustomerID, ode.ProductID, p.ProductName
        FROM "Order Details" ode
        INNER JOIN
            (
            SELECT  COUNT(p.ProductID) ,ode.OrderID, o.CustomerID
            From "Order Details" ode
            INNER JOIN  orders o on ode.OrderID = o.OrderID
            INNER JOIN Products p on ode.ProductID = p.ProductID
            WHERE CustomerId = "ALFKI"
            GROUP BY ode.OrderID
            HAVING COUNT(p.ProductID) > 1) filter
            ON ode.OrderID =filter.OrderID
        INNER JOIN Orders o on o.OrderID = ode.OrderID
        INNER JOIN Products p on ode.ProductID = p.ProductID
        """)

    print "(OrderID, CustomerID, ProductID, ProductName)"
    print "---------------------------------------------"
    
    data = cur.fetchall()
    for d in data:
        print d

print "\nALFKI purchased {} items".format(len(data))

(OrderID, CustomerID, ProductID, ProductName)
---------------------------------------------
(10643, 'ALFKI', 28, 'R\xf6ssle Sauerkraut')
(10643, 'ALFKI', 39, 'Chartreuse verte')
(10643, 'ALFKI', 46, 'Spegesild')
(10702, 'ALFKI', 3, 'Aniseed Syrup')
(10702, 'ALFKI', 76, 'Lakkalik\xf6\xf6ri')
(10835, 'ALFKI', 59, 'Raclette Courdavault')
(10835, 'ALFKI', 77, 'Original Frankfurter gr\xfcne So\xdfe')
(10952, 'ALFKI', 6, "Grandma's Boysenberry Spread")
(10952, 'ALFKI', 28, 'R\xf6ssle Sauerkraut')
(11011, 'ALFKI', 58, 'Escargots de Bourgogne')
(11011, 'ALFKI', 71, 'Flotemysost')

ALFKI purchased 11 items


### Mongo db

In [ ]:
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
from pprint import pprint

mongodb_url = "mongodb://localhost:27017"

# connect to MongoDB
client = MongoClient(mongodb_url)
db=client.Northwind
# Issue the serverStatus command and print the results
serverStatusResult=db.command("serverStatus")
#pprint(serverStatusResult)

In [86]:
orders = db["order-details"].aggregate([
        {
            "$lookup":
                {
                    "from" : "orders",
                    "localField" : "OrderID",
                    "foreignField" : "OrderID",
                    "as" : "order"
                }
        },
        { 
            "$match" : { "order.CustomerID":"ALFKI" }
        },
        {
            "$project" : {"_id":0 ,"OrderID": 1, "ProductID": 1, "order":{"CustomerID":1}, "productInfo":{"ProductName":1}}
        },
        {
            "$group" : {"_id":"$OrderID", "OrderID_counter":{"$sum":1}}
        },
        {
            "$match": {
                "OrderID_counter": {
                    "$gt": 1
                }
            }
        },
        {
            "$lookup":
                {
                    "from" : "order-details",
                    "localField" : "_id",
                    "foreignField" : "OrderID",
                    "as" : "order"
                }
        },
        { 
            "$unwind" : "$order" 
        }
    ])

for order in orders:
    pprint(order)

{u'OrderID_counter': 2,
 u'_id': 11011,
 u'order': {u'Discount': 0,
            u'OrderID': 11011,
            u'ProductID': 71,
            u'Quantity': 20,
            u'UnitPrice': 21.5,
            u'_id': ObjectId('59e520f8899eb088bb819962')}}
{u'OrderID_counter': 2,
 u'_id': 11011,
 u'order': {u'Discount': 0.05,
            u'OrderID': 11011,
            u'ProductID': 58,
            u'Quantity': 40,
            u'UnitPrice': 13.25,
            u'_id': ObjectId('59e520f8899eb088bb819963')}}
{u'OrderID_counter': 2,
 u'_id': 10835,
 u'order': {u'Discount': 0,
            u'OrderID': 10835,
            u'ProductID': 59,
            u'Quantity': 15,
            u'UnitPrice': 55.0,
            u'_id': ObjectId('59e520f8899eb088bb8197af')}}
{u'OrderID_counter': 2,
 u'_id': 10835,
 u'order': {u'Discount': 0.2,
            u'OrderID': 10835,
            u'ProductID': 77,
            u'Quantity': 2,
            u'UnitPrice': 13.0,
            u'_id': ObjectId('59e520f8899eb088bb8197b1')}}